Simple implementation of the Holland & Powell (2011) model. We will need pandas and math.

In [1]:
import pandas as pd
import math
import scipy.optimize as optimize
import timeit

Read in data from Holland & Powell dataset (latest ds61 version, older ds5x version have a slightly different format). First tell the program where to find the dataset:

In [2]:
filename="/home/nick/tc340/mac-version 4/tc-ds62.txt"

Now iterate through the file and populate a pandas dataframe with the parameters we need for the calculations:

In [3]:
hpdata=[]
alldata=[]
names=[]
atoms=[]

with open(filename) as datafile:
    lines=datafile.readlines()
    for i in range(0, len(lines)-3):
        nameline=lines[i]
        stpline=lines[i+1]
        cpline=lines[i+2]
        akline=lines[i+3]
        if len(nameline) > 8 and nameline[7].isalpha():
            nameline=nameline.split()
            atoms=[]
            for j in range(3,len(nameline)):
                if j % 2:
                    atoms.append(float(nameline[j]))
            #nameline=nameline[0]
            #names.append(nameline)
            stpline=stpline.split()
            #stpline=stpline[0],stpline[1],stpline[2]
            #stp.append(stpline)
            cpline=cpline.split()
            #cp.append(cpline)
            akline=akline.split()
            completeline=(stpline[0],stpline[1],stpline[2],cpline[0],cpline[1],cpline[2],cpline[3],akline[0],akline[1],akline[2],akline[3],akline[4],sum(atoms))
            hpdata.append(completeline)
            names.append(nameline[0])
            #ak.append(akline)
            
index = pd.Index(names, name='rows')
columns = pd.Index(["H0","S0","V0","CpA","CpB","CpC","CpD","alpha0","k0","kprime0","kdprime0", "Landau", "atoms"], name='cols')
hpdata = pd.DataFrame(hpdata, index=index, columns=columns)

The relevant parts of the dataset are now in a pandas dataframe called "hpdata", in rows labelled with the abbreviated phase name, and columns with sensible labels. This means that we can call up values such as the bulk modulus in forsterite with a simple command such as: 

In [4]:
print(float(hpdata.loc['q','H0'])-1000*float(hpdata.loc['q','S0']))

-952.15


The next step is to define some functions that will do the calculations. This is analogous to the formulae in the cells of an Excel spreadsheet. First a function to calculate HT-H (as in the contribution of temperature to enthalpy at T, 1 bar), ST-S, and then G(T,1 bar).

In [5]:
def HtminusH(phase,T):
    return float(hpdata.loc[phase,'CpA'])*(T-298.15)+0.5*float(hpdata.loc[phase,'CpB'])*(T*T-298.15*298.15)-float(hpdata.loc[phase,'CpC'])*(1/T-1/298.15)+2*float(hpdata.loc[phase,'CpD'])*(math.sqrt(T)-math.sqrt(298.15))

def StminusS(phase,T):
    return 1000*(float(hpdata.loc[phase,'CpA'])*math.log(T/298.15)+float(hpdata.loc[phase,'CpB'])*(T-298.15)-0.5*float(hpdata.loc[phase,'CpC'])*(1/(T*T)-1/(298.15*298.15))-2*float(hpdata.loc[phase,'CpD'])*(1/math.sqrt(T)-1/math.sqrt(298.15)))

def GTP0(phase,T):
    return ((float(hpdata.loc[phase,'H0'])+HtminusH(phase,T))*1000)-(((float(hpdata.loc[phase,'S0']))*1000)+StminusS(phase,T))*T

def EOSa(phase,P):
     return (1+float(hpdata.loc[phase, 'kprime0']))/((1+float(hpdata.loc[phase, 'kprime0'])+(float(hpdata.loc[phase, 'k0'])*float(hpdata.loc[phase, 'kdprime0']))))

def EOSb(phase,P):
     return (float(hpdata.loc[phase, 'kprime0'])/float(hpdata.loc[phase, 'k0']))-float(hpdata.loc[phase, 'kdprime0'])/(1+float(hpdata.loc[phase, 'kprime0']))

def EOSc(phase,P):
     return (1+float(hpdata.loc[phase, 'kprime0'])+float(hpdata.loc[phase, 'k0'])*float(hpdata.loc[phase, 'kdprime0']))/(float(hpdata.loc[phase, 'kprime0'])*float(hpdata.loc[phase, 'kprime0'])+float(hpdata.loc[phase, 'kprime0'])-float(hpdata.loc[phase, 'k0'])*float(hpdata.loc[phase, 'kdprime0']))

def Pth(phase,T):
    theta=10636/(((float(hpdata.loc[phase, 'S0'])*1000)/float(hpdata.loc[phase, 'atoms']))+6.44)
    xi0=(theta/298.15)*(theta/298.15)*math.exp(theta/298.15)/((math.exp(theta/298.15)-1)*(math.exp(theta/298.15)-1))
    Pthermal=float(hpdata.loc[phase, 'alpha0'])*float(hpdata.loc[phase, 'k0'])*theta/xi0*(1/(math.exp(theta/T)-1)-1/(math.exp(theta/298.15)-1))
    return(Pthermal)

def V(phase,P,T):
    return float(hpdata.loc[phase, 'V0'])*(1-EOSa(phase,P)*(1-math.pow((1+EOSb(phase,P)*(P-Pth(phase,T))),(-EOSc(phase,P)))))

def PVcorr(phase,P,T):
    return 1-EOSa(phase,P)+(EOSa(phase,P)*(math.pow((1-EOSb(phase,P)*Pth(phase,T)),(1-EOSc(phase,P)))-math.pow((1+EOSb(phase,P)*(P-Pth(phase,T))),(1-EOSc(phase,P))))/(EOSb(phase,P)*(EOSc(phase,P)-1)*P))

def GPTminusGP0T(phase,P,T):
    return PVcorr(phase,P,T)*P*float(hpdata.loc[phase, 'V0'])

def GPT(phase,P,T):
    return GTP0(phase,T)+GPTminusGP0T(phase,P,T)*1000

G for any phase at conditions P,T can now be easily calculated by calling these functions:

In [6]:
print(GPT('q',30,1773.15))
print(GTP0('q',298.15))

-1017238.8248047617
-923072.3545


We can use this to find out whether the products or reactants of a reaction will be the equilibrium assemblage at a given P,T.

Simple example: kyanite=sillimanite=andalusite at 550 °C, 4 kbar.

In [7]:
T=550+273.15 # temperature in Kelvin
P=4 # H&P dataset is in kbar rather than GPa

print('Kyanite:', GPT('ky',P,T))
print('Andalusite:', GPT('and',P,T))
print('Sillimanite:', GPT('sill',P,T))

Kyanite: -2691258.325361273
Andalusite: -2691610.170193039
Sillimanite: -2691570.863185545


Because this P,T is close to the the invariant triple point at which all of the polymorphs are stable, these values for G(P,T) are close together. The minimum value is that for kyanite, indicating that these conditions are in the kyanite stability field.

This can be used to find univariant points, ie. phase boundaries if P or T is fixed. We can try this with the kyanite=andalusite boundary at 550 °C.

In [8]:
T=550+273.15 # T in K

# define a function that takes only the variable we are interested in that will calculate DeltaG across the reaction.
def andkycalc(P):
    return GPT('ky',P, 550+273)-GPT('and',P,550+273)

Quick test to find DeltaG at 5 kbar:

In [9]:
andkycalc(5)

-389.212613188196

We can use the Newton-Raphson to find the zero point for the and=ky function, giving a nearby starting point.

In [10]:
optimize.newton(andkycalc, x0=5)

4.473662260185149

We can wrap this up in a function that takes the name of the phase on either side of the reaction, temperature and a pressure guess:

In [13]:
def univariantPseek(Pguess,T,phase1,phase2):
    def univariantPcalc(P,T,phase1,phase2):
        return GPT(phase1,P,T)-GPT(phase2,P,T)
    return optimize.newton(univariantPcalc, x0=Pguess, args=(T, phase1, phase2))

In [14]:
print('Ptr ky=and at 773 K:', univariantPseek(5,773,'ky','and'), 'kbar')

Ptr ky=and at 773 K: 3.8576879334653964 kbar


To use this for reactions involving quartz, we will need to add in Landau order-disorder, to model the second-order alpha=beta transition. Without this, the calculations are not particularly useful. Eg:

In [15]:
print('Ptr q=coe at 1273.15 K:', univariantPseek(30,1273.15,'q','coe'),'kbar')

Ptr q=coe at 1273.15 K: 29.87599699587954 kbar


Bose & Ganguly (1995) bracketed Ptr q=coe at 1073K betwee 30.6 and 30.9 kbar, so this is out by ~1 kbar.

We can also define a function to find Ttr at given P:

In [ ]:
def univariantTseek(Tguess,P,phase1,phase2):
    def univariantTcalc(T,P,phase1,phase2):
        return GPT(phase1,P,T)-GPT(phase2,P,T)
    return optimize.newton(univariantTcalc, x0=Tguess, args=(P, phase1, phase2))

# check this with ky=sill at 7 kbar.
print('Ttr ky=sill at 7 kbar:', univariantTseek(900,7,'ky','sill')-273.15, '°C')

Define a function to find pressure and temperature of an invariant point at which 3 phases are present (eg. ky=and=sill). This will contain a calculation function that takes pressure and temperature as an array.

In [ ]:
def invariantPTseek(PTguess,phase1,phase2,phase3):
    def invariantPTsumsq(PTguess,phase1,phase2,phase3):
        P,T=PTguess
        eq1=GPT(phase1,P,T)-GPT(phase2,P,T)
        eq2=GPT(phase2,P,T)-GPT(phase3,P,T)
        eq3=GPT(phase1,P,T)-GPT(phase3,P,T)
        return eq1*eq1+eq2*eq2+eq3*eq3
    return optimize.minimize(invariantPTsumsq, PTguess, args=(phase1, phase2, phase3), method='Nelder-Mead')

In [ ]:
invariantPTseek([5,500],'ky','and','sill')

Let's see how long this takes:

In [ ]:
%time pt=invariantPTseek([5,500],'ky','and','sill')

print("Univariant point at", pt.x[0], "kbar", pt.x[1]-273, "°C")

In [ ]:
print(HtminusH('fo',1373))
print(StminusS('fo',1373))
print(GTP0('fo',1373))
print(Pth('fo',1373))
print(V('fo',10,1373))